In [4]:
import pandas as pd
import numpy as np
from pyfixest import feols
import statsmodels.api as sm

# Load data
clean_wdf = pd.read_csv('hsg_schedules_2022_2025.csv')

# Define variable groups
null = ['time_to_next', 'caseload', 'no_cases']
enhanced_null_prev_assigned = null + ['previously_assigned']
enhanced_null_prev_assigned_ij_throughout = null + ['previously_assigned', 'ij_throughout']
priority_dockets = ['length_of_proceedings', 'custody_N', 'casepriority__UC', 'family']

# Categorical dummies
columns = list(clean_wdf.columns)
lang = [col for col in columns if 'lang_' in col]
nat = [col for col in columns if 'nat_' in col]
casetype = [col for col in columns if 'casetype_' in col]
custody = [col for col in columns if 'custody_' in col]

# Model variable combinations
model_variables = { 
    "Null": null, 
    "Enhanced Null": enhanced_null_prev_assigned, 
    "Enhanced Null IJ Throughout": enhanced_null_prev_assigned_ij_throughout, 
    "Null and Priority Dockets": null + priority_dockets,
    "Enhanced Null and Priority Dockets": enhanced_null_prev_assigned + priority_dockets,
    "Enhanced Null IJ Throughout and Priority": enhanced_null_prev_assigned_ij_throughout + priority_dockets,
    "Null and Language": null + lang,
    "Null and Nationality": null + nat,
    "Null and Case Type": null + casetype,
    "Null and Custody": null + custody,
    "Enhanced Null and Custody": enhanced_null_prev_assigned + custody,
    "Enhanced Null and Language": enhanced_null_prev_assigned + lang,
    "Enhanced Null and Nationality": enhanced_null_prev_assigned + nat,
    "Enhanced Null and Case Type": enhanced_null_prev_assigned + casetype,
    "Enhanced Null with Priority and Language": enhanced_null_prev_assigned + priority_dockets + lang,
    "Enhanced Null with Priority and Nationality": enhanced_null_prev_assigned + priority_dockets + nat,
    "Enhanced Null with Priority and Case Type": enhanced_null_prev_assigned + priority_dockets + casetype,
    "Enhanced Null with Priority and Custody": enhanced_null_prev_assigned + priority_dockets + custody,
    "Full Model (No Lawyer)": enhanced_null_prev_assigned_ij_throughout + priority_dockets + lang + nat + casetype + custody,
    "Kitchen Sink": enhanced_null_prev_assigned_ij_throughout + priority_dockets + lang + nat + casetype + custody + ['lawyer']
}

# Fix datetime parsing - handle milliseconds with format='mixed'
try:
    clean_wdf['scheduled_datetime'] = pd.to_datetime(clean_wdf['scheduled_datetime'], format='mixed')
except Exception:
    # If that fails, try a different approach
    try:
        clean_wdf['scheduled_datetime'] = pd.to_datetime(clean_wdf['scheduled_datetime'], infer_datetime_format=True, errors='coerce')
    except Exception:
        # If all else fails, create a simpler time variable from components if available
        if 'scheduled_year' in clean_wdf.columns and 'scheduled_month' in clean_wdf.columns:
            clean_wdf['time_fe'] = clean_wdf['scheduled_year'].astype(str) + '-' + clean_wdf['scheduled_month'].astype(str).str.zfill(2)
        else:
            # Extract year from string if possible as fallback
            clean_wdf['time_fe'] = clean_wdf['scheduled_datetime'].str.extract(r'(\d{4})', expand=False)

# Create time fixed effect variables
# Only continue if datetime parsing succeeded
if pd.api.types.is_datetime64_dtype(clean_wdf['scheduled_datetime']):
    # Year fixed effects - simplest option
    clean_wdf['scheduled_year'] = clean_wdf['scheduled_datetime'].dt.year
    
    # Half-year fixed effects
    clean_wdf['scheduled_half_year'] = (clean_wdf['scheduled_datetime'].dt.year.astype(str) + 'H' + 
                                       ((clean_wdf['scheduled_datetime'].dt.month - 1) // 6 + 1).astype(str))
    
    # Quarter fixed effects
    clean_wdf['scheduled_quarter'] = (clean_wdf['scheduled_datetime'].dt.year.astype(str) + 'Q' + 
                                     ((clean_wdf['scheduled_datetime'].dt.month - 1) // 3 + 1).astype(str))
    
    # Month fixed effects - more granular, might cause convergence issues
    clean_wdf['scheduled_month_year'] = (clean_wdf['scheduled_datetime'].dt.year.astype(str) + '-' + 
                                        clean_wdf['scheduled_datetime'].dt.month.astype(str).str.zfill(2))
    
    # Use scheduled_year as the default time FE if the more granular ones cause problems
    time_fe_var = 'scheduled_year'
else:
    # If datetime conversion failed, use time_fe as created in the exception handling
    time_fe_var = 'time_fe'

# Check if the time fixed effect variable exists
if time_fe_var not in clean_wdf.columns:
    # Create a dummy time fixed effect as fallback
    clean_wdf['time_fe'] = 1
    time_fe_var = 'time_fe'
    print("Warning: Could not create proper time fixed effects. Using dummy variable instead.")

# Run the models with fixed effects using pyfixest
fe_models = {}

# Function to run models with progressively simpler fixed effects
def run_model_with_fallbacks(title, x_variables, data, y_var='assigned'):
    # List of time FE variables to try in order of complexity
    # Start with the most granular and fall back to simpler ones if needed
    time_fe_options = [
        'scheduled_month_year',  # Most granular: month-year
        'scheduled_quarter',     # Quarterly
        'scheduled_half_year',   # Half-yearly
        'scheduled_year',        # Yearly
        'time_fe'                # Fallback dummy
    ]
    
    # Try each time FE option until one works
    for time_fe in time_fe_options:
        if time_fe not in data.columns:
            continue
            import delimited MIA_fe_ready.csv;
        try:
            print(f"\nTrying model '{title}' with time FE: {time_fe}")
            
            # Create the formula - with one variable at a time to avoid string manipulation issues
            formula_parts = [y_var, '~']
            
            for var in x_variables:
                if var in data.columns:  # Only include variables that exist in the dataframe
                    formula_parts.append(var)
                    formula_parts.append('+')
            
            if formula_parts[-1] == '+':
                formula_parts.pop()  # Remove trailing '+' if present
                
            formula_parts.append('|')
            formula_parts.append(time_fe)
            
            formula = ' '.join(formula_parts)
            print(f"Formula: {formula}")
            
            # Run the model with more relaxed tolerance settings
            model = feols(
                formula, 
                data=data,
                fixef_tol=1e-4,     # More relaxed tolerance
                collin_tol=1e-5,    # More relaxed collinearity tolerance
            )
            
            print(f"Success with {time_fe} fixed effects!")
            return model, time_fe
            
        except Exception as e:
            print(f"Error with {time_fe}: {str(e)[:100]}...")
    
    # If all fixed effects approaches fail, fall back to OLS
    print(f"\nAll fixed effects approaches failed for '{title}'. Falling back to OLS.")
    
    valid_x_vars = [var for var in x_variables if var in data.columns]
    X = sm.add_constant(data[valid_x_vars])
    y = data[y_var]
    
    try:
        ols_model = sm.OLS(y, X).fit()
        return ols_model, None
    except Exception as e:
        print(f"Even OLS failed: {str(e)[:100]}...")
        return None, None

# Run all models
results = {}
for title, x_variables in model_variables.items():
    print(f"\n{'='*50}")
    print(f"Starting model: {title}")
    print(f"{'='*50}")
    
    model, fe_used = run_model_with_fallbacks(title, x_variables, clean_wdf)
    
    if model is not None:
        fe_models[title] = model
        results[title] = {
            'model': model,
            'time_fe_used': fe_used
        }
        
        # Print summary
        print(f"\n{'='*25}  {title.upper()}  {'='*25}\n")
        print(f"Time Fixed Effect used: {fe_used if fe_used else 'None (OLS)'}")
        print(model.summary())
    else:
        print(f"Could not estimate model for '{title}'")

# Print a summary of which fixed effect was used for each model
print("\n\n")
print("="*80)
print("SUMMARY OF MODELS AND FIXED EFFECTS USED")
print("="*80)
for title, result in results.items():
    fe = result['time_fe_used'] if result['time_fe_used'] else "None (OLS)"
    print(f"{title:50s}: {fe}")


Starting model: Null

Trying model 'Null' with time FE: scheduled_month_year
Formula: assigned ~ time_to_next + caseload + no_cases | scheduled_month_year
Success with scheduled_month_year fixed effects!

=========================  NULL  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next  |      0.000 |        0.000 |    10.452 |      0.000 |  0.000 |   0.000 |
| caseload      |      0.000 |        0.000 |     9.972 |      0.000 |  0.000 |   0.000 |
| no_cases      |     -0.038 |        0.006 |    -5.888 |      0.000 | -0.051 |  -0.025 |
---
RMSE: 0.257 R2: 0.026 R2 Within: 0.024 
None

Starting model: Enhanced Null

Trying model 'Enhanced Null' with time 

/home/shuba/.local/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2545: UserWarning: 
            The following variables are collinear: ['lang_other'].
            The variables are dropped from the model.
            
  warnings.warn(


Success with scheduled_month_year fixed effects!

=========================  NULL AND LANGUAGE  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next  |      0.000 |        0.000 |    10.453 |      0.000 |  0.000 |   0.000 |
| caseload      |      0.000 |        0.000 |     9.972 |      0.000 |  0.000 |   0.000 |
| no_cases      |     -0.038 |        0.006 |    -5.888 |      0.000 | -0.051 |  -0.025 |
| lang_AR       |      0.000 |        0.000 |     1.290 |      0.205 | -0.000 |   0.001 |
| lang_ENG      |      0.000 |        0.000 |     0.252 |      0.802 | -0.000 |   0.000 |
| lang_MAN      |      0.000 |        0.000 |     0.039 |      0.969 | -0.001 |   0

/home/shuba/.local/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2545: UserWarning: 
            The following variables are collinear: ['nat_other'].
            The variables are dropped from the model.
            
  warnings.warn(


Success with scheduled_month_year fixed effects!

=========================  NULL AND NATIONALITY  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next  |      0.000 |        0.000 |    10.453 |      0.000 |  0.000 |   0.000 |
| caseload      |      0.000 |        0.000 |     9.972 |      0.000 |  0.000 |   0.000 |
| no_cases      |     -0.038 |        0.006 |    -5.888 |      0.000 | -0.051 |  -0.025 |
| nat_CU        |     -0.000 |        0.000 |    -1.465 |      0.152 | -0.000 |   0.000 |
| nat_ES        |     -0.000 |        0.000 |    -1.100 |      0.279 | -0.000 |   0.000 |
| nat_HO        |     -0.000 |        0.000 |    -1.387 |      0.174 | -0.000 | 

...

Success with scheduled_month_year fixed effects!

=========================  ENHANCED NULL WITH PRIORITY AND CUSTODY  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient           |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:----------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next          |      0.000 |        0.000 |    10.431 |      0.000 |  0.000 |   0.000 |
| caseload              |      0.000 |        0.000 |     9.969 |      0.000 |  0.000 |   0.000 |
| no_cases              |     -0.038 |        0.006 |    -5.889 |      0.000 | -0.051 |  -0.025 |
| previously_assigned   |      0.001 |        0.000 |     3.105 |      0.004 |  0.000 |   0.002 |
| length_of_proceedings |      0.000 |        0.000 |     2.049 |      0.048 |  0.000 |   0.000 |
| cus

/home/shuba/.local/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2545: UserWarning: 
            The following variables are collinear: ['lang_other', 'nat_other', 'casetype_WHO', 'custody_R'].
            The variables are dropped from the model.
            
  warnings.warn(


Success with scheduled_month_year fixed effects!

=========================  FULL MODEL (NO LAWYER)  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient           |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:----------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next          |      0.000 |        0.000 |    10.432 |      0.000 |  0.000 |   0.000 |
| caseload              |      0.000 |        0.000 |     9.968 |      0.000 |  0.000 |   0.000 |
| no_cases              |     -0.038 |        0.006 |    -5.889 |      0.000 | -0.051 |  -0.025 |
| previously_assigned   |      0.001 |        0.000 |     3.036 |      0.004 |  0.000 |   0.002 |
| length_of_proceedings |      0.000 |        0.000 |     2.112 |      0.042 |  0.000 |   0.000 |
| custody_N           

/home/shuba/.local/lib/python3.10/site-packages/pyfixest/estimation/feols_.py:2545: UserWarning: 
            The following variables are collinear: ['lang_other', 'nat_other', 'casetype_WHO', 'custody_R'].
            The variables are dropped from the model.
            
  warnings.warn(


Success with scheduled_month_year fixed effects!

=========================  KITCHEN SINK  =========================

Time Fixed Effect used: scheduled_month_year
###

Estimation:  OLS
Dep. var.: assigned, Fixed effects: scheduled_month_year
Inference:  CRV1
Observations:  343157

| Coefficient           |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:----------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| time_to_next          |      0.000 |        0.000 |    10.432 |      0.000 |  0.000 |   0.000 |
| caseload              |      0.000 |        0.000 |     9.968 |      0.000 |  0.000 |   0.000 |
| no_cases              |     -0.038 |        0.006 |    -5.889 |      0.000 | -0.051 |  -0.025 |
| previously_assigned   |      0.001 |        0.000 |     3.035 |      0.004 |  0.000 |   0.002 |
| length_of_proceedings |      0.000 |        0.000 |     2.079 |      0.045 |  0.000 |   0.000 |
| custody_N             |      0

In [6]:
clean_wdf.to_csv('hsg_scheduling_2022.csv')

In [7]:
clean_wdf

,Unnamed: 0,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,...,nat_MX,nat_VE,nat_other,previous_assigned_judge,previously_assigned,family,scheduled_year,scheduled_half_year,scheduled_quarter,scheduled_month_year
0,0,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,...,0,0,1,NaN,0,0,2022,2022H1,2022Q2,2022-05
1,1,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,...,0,0,1,NaN,0,0,2022,2022H1,2022Q2,2022-05
2,2,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,...,0,0,1,NaN,0,0,2022,2022H1,2022Q2,2022-05
3,3,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,...,0,0,1,NaN,0,0,2022,2022H1,2022Q2,2022-05
4,4,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,...,0,0,1,NaN,0,0,2022,2022H1,2022Q2,2022-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343152,343152,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,...,0,0,0,NaN,0,1,2025,2025H1,2025Q1,2025-02
343153,343153,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,...,0,0,0,NaN,0,1,2025,2025H1,2025Q1,2025-02
343154,343154,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,...,0,0,0,NaN,0,1,2025,2025H1,2025Q1,2025-02
343155,343155,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,...,0,0,0,NaN,0,1,2025,2025H1,2025Q1,2025-02
